In [43]:
import mysql.connector as connector
import pandas as pd

In [44]:
try:
    connection = connector.connect(
    user='root',
    password='******'
)
    print('Connected...')
except connector.Error as err:
    print('Error code: ', err.errno)
    print('Error msg: ', err.msg)

Connected...


In [45]:
try:
    cursor = connection.cursor()
    print('Cursor created...')
except:
    print('Its not possible to create the cursor!')

Cursor created...


In [46]:
cursor.execute("USE meta_py_to_mysql")
print('Database in use: ', connection.database)

Database in use:  meta_py_to_mysql


In [8]:
# 1. Create a stored procedure (without parameters) that make a join between 'bookings' and 'orders' tables to output the bill amount and the customer name:

cursor.execute("DROP PROCEDURE IF EXISTS getCustomersAndBillAmount") # Make sure the procedure do not exists before

stored_procedure_query = """
CREATE PROCEDURE getCustomersAndBillAmount()
BEGIN
SELECT b.bookingID, 
CONCAT(b.guestFirstName, ' ', b.guestLastName) AS customerName,
o.billAmount
FROM bookings b INNER JOIN orders o ON b.bookingID = o.bookingID 
ORDER BY billAmount DESC;
END
"""

cursor.execute(stored_procedure_query)
print("Procedure 'getCustomersAndBillAmount' created!")


Procedure 'getCustomersAndBillAmount' created!


In [21]:
# 1.1. Call the created procedure:

cursor.callproc("getCustomersAndBillAmount")
results1 = next(cursor.stored_results()) # next() function returns the next item in a list (it's like "manual for loop")
dataset1 = results1.fetchall()
columns1 = results1.column_names

df1 = pd.DataFrame(data=dataset1, columns=columns1)
df1.style.hide(axis='index')

bookingID,customerName,billAmount
1,Anna Iversen,2
2,Joakim Iversen,1
3,Vanessa McCarthy,1
5,Hiroki Yamane,1


In [58]:
# 2. Retrieve the booking ID, guest’s full name, and the bill amount of the top spender at closing.

cursor.execute("DROP PROCEDURE IF EXISTS TopSpender;")

procedure_topSpender = """
CREATE PROCEDURE TopSpender()
BEGIN
SELECT b.bookingID, 
CONCAT(b.guestFirstName,' ',b.guestLastName) AS name,
 MAX(o.billAmount) AS bill
FROM bookings b
INNER JOIN orders o
ON b.bookingID = o.bookingID;
END
"""
cursor.execute(procedure_topSpender)
print('Procedure created...')

Procedure created...


In [59]:
# 2.1. Call Stored procedure:

cursor.callproc("TopSpender")
results2 = next(cursor.stored_results())
dataset2 = results2.fetchall()
columns2 = results2.column_names

df2 = pd.DataFrame(data=dataset2, columns= columns2)
df2.style.hide(axis='index')

bookingID,name,bill
1,Anna Iversen,2


In [60]:
# 3. Create a procedure that retrieve all bookings where the guests did not apper in today orders:

cursor.execute("""DROP PROCEDURE IF EXISTS NoArrival;""")

query_NoArrival ="""
CREATE PROCEDURE NoArrival()
BEGIN
SELECT bookingID, 
CONCAT(guestFirstName,' ',guestLastName) AS name,
tableNo
FROM bookings 
WHERE bookingID NOT IN (SELECT bookingID FROM orders);
END
"""
cursor.execute(query_NoArrival)
print('Procedure created...')

Procedure created...


In [61]:
# 3.1. Call Stored procedure:

cursor.callproc("NoArrival")
results3 = next(cursor.stored_results())
dataset3 = results3.fetchall()
columns3 = results3.column_names

df3 = pd.DataFrame(data=dataset3, columns=columns3)
df3.style.hide(axis='index')

bookingID,name,tableNo
6,Diana Pinto,10


In [55]:
# 4. Categorize orders:

"""
This is how they categorize the orders: 

* If not assigned to any employee, the status is `In Queue` 
* If assigned to the employees with IDs 4 and 5, the status is `Preparing Order` 
* If assigned to the employees with IDs 1, 2, and 3, the status is `Order Served` 
"""

cursor.execute("""DROP PROCEDURE IF EXISTS OrderStatus;""")

query_OrderStatus = """

CREATE PROCEDURE OrderStatus()

BEGIN
SELECT bookingID, 
CONCAT(guestFirstName,' ',guestLastName) AS name,
CASE 
    WHEN employeeID IS NULL THEN 'In Queue'
    WHEN employeeID IN (4,5) THEN 'Preparing Order'
    WHEN employeeID IN (1,2,3) THEN 'Order Served'
END AS Status
FROM bookings;

END
"""

cursor.execute(query_OrderStatus)
print('Procedure created...')

Procedure created...


In [57]:
# 4.1. Call the created procedure:

cursor.callproc('OrderStatus')
results4 = next(cursor.stored_results())
dataset4 = results4.fetchall()
columns4 = results4.column_names

df4 = pd.DataFrame(data=dataset4, columns=columns4)
df4.style.hide(axis='index')

bookingID,name,Status
1,Anna Iversen,Order Served
2,Joakim Iversen,Order Served
3,Vanessa McCarthy,Order Served
5,Hiroki Yamane,Order Served
6,Diana Pinto,Preparing Order


In [70]:
# 5. Procedure with one argument:

cursor.execute('DROP PROCEDURE IF EXISTS itemID_price')

query_itemID_price = """
CREATE PROCEDURE itemID_price(IN item_id INT, OUT price INT)

BEGIN
SELECT * FROM menu_items WHERE itemID = item_id;
END
"""
cursor.execute(query_itemID_price)
print('Procedure created...')

Procedure created...


In [74]:
# 5.1 Call created procedure passing one argument:
# The objective is to pass the ItemID of an item in the item_menu table and get all the information about.

args = (2,0) # the 0 is to hold value of the OUT parameter but the 2 is the IN argument
cursor.callproc('itemID_price', args)
results5 = next(cursor.stored_results())
dataset5 = results5.fetchall()
columns5 = results5.column_names

df5 = pd.DataFrame(data= dataset5, columns= columns5)
df5.style.hide(axis='index')

itemID,name,type,price
2,Flatbread,Starters,5
